In [1]:
import numpy as np
import pandas as pd
import requests
import time

In [2]:
api_key = "6bb7f55c691d1bdff841fdc67934b3b8"

### Find Song

In [4]:
def find_song(artist, track):
    results = []
    artists_list = []
    params_get_artists = {
        "method" : "artist.search",
        "format" : "json",
        "limit" : 100,
        "artist" : artist,
        "api_key" : api_key
    }
    artists = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_get_artists)
    for artist in artists.json()['results']['artistmatches']['artist']:
            artists_list.append(artist['name'])

    tracks_list = []
    params_get_tracks = {
        "method" : "track.search",
        "format" : "json",
        "limit" : 100,
        "track" : track,
        "api_key" : api_key
    }
    tracks = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_get_tracks)
    for track in tracks.json()['results']['trackmatches']['track']:
        if track['artist'] in artists_list:
            return_track = track['name']
            return_artist = track['artist']
            results.append((return_artist, return_track))
    if len(results) > 0:
        return results
    return('Could not find track. Please search again.')

## Get Results

In [92]:
results = find_song(artist = 'nsync', track = "bye")
result_artist = results[0][0]
result_track = results[0][1]
print(result_artist)
print(result_track)

*NSYNC
Bye Bye Bye


## Get Artist Info

In [74]:
def get_artist_info(result_artist):
    params_artist_info = {
    "method" : "artist.getInfo",
    "format" : "json",
    "artist" : result_artist,
    "api_key" : api_key
}
    artist_info = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_artist_info)
    content = artist_info.json()['artist']['bio']['content']
    listeners = artist_info.json()['artist']['stats']['listeners']
    playcount = artist_info.json()['artist']['stats']['playcount']
    print(f'Listeners: {listeners}')
    print('')
    print(f'Playcount: {playcount}')
    print('\n')
    print(f'Artist Info: {content}')

In [93]:
get_artist_info(result_artist)

Listeners: 1028153

Playcount: 13870752


Artist Info: *NSYNC was an American pop boy band that is widely regarded as one of the most successful pop acts of the late '90s and early 2000s, selling over 56 million records worldwide. Formed in Orlando, Florida in 1995, the group consists of Lance Bass, JC Chasez, Joey Fatone, Chris Kirkpatrick and Justin Timberlake. 

The band released three regular studio albums, *NSYNC (1998), No Strings Attached (2000), and Celebrity (2001), as well as the full length Christmas album Home for Christmas (1998).

The group went on an extended hiatus in spring 2002, and only Timberlake and Chasez have gone on to release solo albums. <a href="https://www.last.fm/music/%2ANSYNC">Read more on Last.fm</a>. User-contributed text is available under the Creative Commons By-SA License; additional terms may apply.


## Get Track Info

In [76]:
def get_track_info(result_artist, result_track):
    params_track_info = {
    "method" : "track.getInfo",
    "format" : "json",
    "artist" : result_artist,
    "track" : result_track,
    "api_key" : api_key
}
    track_info = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_track_info)
    try:
        track_content = track_info.json()['track']['wiki']['content']
    except:
        pass
    duration = track_info.json()['track']['duration']
    if int(duration) > 0:
        print(f"Duration: {duration}")
    print(f"Listeners: {track_info.json()['track']['listeners']}")
    print(f"Playcount: {track_info.json()['track']['playcount']}")
    try:
        print(f"Album: {track_info.json()['track']['album']['title']}")
    except:
        pass

In [94]:
get_track_info(result_artist, result_track)

Duration: 199000
Listeners: 452208
Playcount: 2159606
Album: No Strings Attached


## Get Similar Tracks

In [82]:
def get_similar_tracks(result_artist, result_track):
    params_get_similar_tracks = {
        "method" : "track.getSimilar",
        "format" : "json",
        "artist" : result_artist,
        "track" : result_track,
        "api_key" : api_key
    }
    similar_tracks = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_get_similar_tracks)
    counter = 1
    for track in similar_tracks.json()['similartracks']['track'][:10]:
        return_similar_track = track['name']
        return_similar_artist = track['artist']['name']
        print(f'Track: {return_similar_track}')
        print(f'Artist: {return_similar_artist}')
        get_track_info(return_similar_artist, return_similar_track)
        print('')

In [95]:
get_similar_tracks(result_artist, result_track)

Track: It's Gonna Be Me
Artist: *NSYNC
Duration: 191000
Listeners: 252080
Playcount: 1302937
Album: No Strings Attached

Track: I Want You Back
Artist: *NSYNC
Duration: 200000
Listeners: 76335
Playcount: 444886
Album: *NSYNC

Track: Larger Than Life
Artist: Backstreet Boys
Duration: 232000
Listeners: 266136
Playcount: 1267167
Album: Millennium

Track: I Want It That Way
Artist: Backstreet Boys
Duration: 214000
Listeners: 645341
Playcount: 3503180
Album: Millennium

Track: Genie in a Bottle
Artist: Christina Aguilera
Duration: 216000
Listeners: 609378
Playcount: 2849360
Album: Christina Aguilera

Track: Toxic
Artist: Britney Spears
Duration: 199000
Listeners: 992536
Playcount: 7246376
Album: In the Zone

Track: Wannabe
Artist: Spice Girls
Duration: 172000
Listeners: 508507
Playcount: 2476020
Album: Spice

Track: ...Baby One More Time
Artist: Britney Spears
Duration: 210000
Listeners: 661521
Playcount: 4250267
Album: Greatest Hits: My Prerogative

Track: Spice Up Your Life
Artist: Spice 

## Get Similar Artists

In [84]:
def get_artist_counts(result_artist):
    params_artist_counts = {
    "method" : "artist.getInfo",
    "format" : "json",
    "artist" : result_artist,
    "api_key" : api_key
}
    artist_info = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_artist_counts)
    listeners = artist_info.json()['artist']['stats']['listeners']
    playcount = artist_info.json()['artist']['stats']['playcount']
    print(f'Artist: {result_artist}')
    print('')
    print(f'Listeners: {listeners}')
    print('')
    print(f'Playcount: {playcount}')
    print('\n')

In [85]:
def get_similar_artists(result_artist):
    params_get_similar_artists = {
        "method" : "artist.getSimilar",
        "format" : "json",
        "artist" : result_artist,
        "api_key" : api_key
    }
    similar_artists = requests.get('http://ws.audioscrobbler.com/2.0/', params=params_get_similar_artists)
    counter = 1
    for artist in similar_artists.json()['similarartists']['artist'][:10]:
        return_similar_artist_art = artist['name']
        get_artist_counts(return_similar_artist_art)

In [96]:
get_similar_artists(result_artist)

Artist: Backstreet Boys

Listeners: 1525922

Playcount: 33609922


Artist: O-Town

Listeners: 138613

Playcount: 982488


Artist: JC Chasez

Listeners: 85646

Playcount: 641688


Artist: Five

Listeners: 308046

Playcount: 2630663


Artist: Justin Timberlake

Listeners: 2959459

Playcount: 78090898


Artist: 98 Degrees

Listeners: 155449

Playcount: 1233691


Artist: 98º

Listeners: 85839

Playcount: 421899


Artist: New Kids on the Block

Listeners: 372565

Playcount: 3429813


Artist: Jessica Simpson

Listeners: 668158

Playcount: 5688941


Artist: BBMak

Listeners: 142900

Playcount: 1017276




## twitter API